# Trabajo de Inteligencia Artificial

## Análisis de sentimientos

Realizado por:
- Alicia Sánchez Hossdorf
- Rafael Segura Gómez

Fecha: 26/05/2023

Convocatoria de Junio

<h1> 1. Recopilación de datos. </h1>

In [256]:
import pandas
import numpy

Hemos descargado todos los tweets que necesitamos en un CSV. Por lo que debemos leer los datos del archivo.

In [257]:
tweetsCSV = pandas.read_csv("../Notebook/Tweets/Tweets_entrenamiento.csv", header=0)


Para saber cuantos tweets tenemos, miraremos cuantas filas tiene el archivo.

In [258]:
print(tweetsCSV.shape)

(27480, 4)


In [304]:
# 10 primeras filas
tweetsCSV.head(5)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


Como solo necesitamos la columna text, que es donde viene los tweets que necesitamos, haremos una seleccion de dicha columna.


In [260]:
tweets = tweetsCSV[['text']]

In [261]:
tweets.head(10)

,text
0,"I`d have responded, if I were going"
1,Sooo SAD I will miss you here in San Diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on t..."
5,http://www.dothebouncy.com/smf - some shameles...
6,2am feedings for the baby are fun when he is a...
7,Soooo high
8,Both of you
9,Journey!? Wow... u just became cooler. hehe....


<h1> 2. Eliminar las palabras que no aportan información </h1>

In [262]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [263]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [264]:
from nltk import download

download('punkt', download_dir='.')

[nltk_data] Downloading package punkt to ....
[nltk_data]   Package punkt is already up-to-date!


True

In [265]:
sWords = set(stopwords.words('english'))
print(sWords)

{'against', "you've", 'his', 'himself', 'most', 'such', 'while', 'our', "you'll", 'wouldn', 'mustn', 'in', 'after', 's', 'isn', "won't", 'off', 'your', 'there', 'don', "haven't", 'over', 'for', 'further', 'ours', "mightn't", "isn't", 'what', 'own', "you're", 'now', 'only', 'itself', 'not', 'and', 'each', 'here', 'how', 'won', 'is', 'same', "should've", 'hasn', 'out', "don't", "doesn't", "she's", 'its', 'but', 'hadn', 'any', 't', 'be', 'didn', 've', 'she', 'through', 'did', 'with', 'from', 'other', 'where', 'themselves', 'been', 'of', 'during', "aren't", 'up', "hadn't", 'my', 'am', 'has', 'doing', "you'd", "that'll", 'yourself', 'than', 'weren', 'about', 'having', 'because', 'needn', 'under', 'before', 'when', 'why', 'both', 'those', 'too', 'again', 'doesn', 'whom', 'their', 'that', 'does', 'had', 'into', 'can', 'down', 'ain', 'mightn', "weren't", "wouldn't", 'so', 'her', 'until', 'haven', 'or', 'few', 'ma', "didn't", 'nor', 'all', 'them', 'more', 'couldn', 'these', 'wasn', 'are', 'belo

Crear una función limpiar_texto para eliminar las palabras comunes y poco informativas; eliminar menciones, hashtags, URLs y cualquier otro símbolo extraño; y que utilice técnicas de lematización (stemming) y tokenización para reducir las palabras a su forma base

In [276]:
#Funcion para eliminar las stop words.

def eliminar_stopWords(tweet):
    tweetMin = tweet.lower()
    tweetWords = word_tokenize(tweetMin)
    tweetFiltered = []
        
    for w in tweetWords:
        if w not in sWords:
            tweetFiltered.append(w)
    return tweetFiltered

In [301]:
#Esta funcion es para eliminar menciones, hashtags, URLs y cualquier otro símbolo extraño.

def eliminar_otrosElementos(tweet):
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("http[^\s]+","",tweet)
    tweetFiltered = re.sub(r"[-()\"#/@;:<>{}`+=*~|.!?,]", "", tweet)

    return tweetFiltered
    

## Función para limpiar el texto

In [302]:
def limpiar_texto(Tweets): 
    res = Tweets.copy()
    i = 0;
    while i<len(tweets):
        tweet_limpios = ""
        clean_t = eliminar_otrosElementos(Tweets['text'].get(i))
        tSinStopWords = eliminar_stopWords(clean_t)
        #Esto es para la lematización de la frase. 
        for w in tSinStopWords:
            tweet_limpios = tweet_limpios + PorterStemmer().stem(w) + " "
        res.at[i,'text'] = tweet_limpios
        
        i+=1
    return res

In [303]:
limpiar_texto(tweets)

,text
0,id respond go
1,sooo sad miss san diego
2,boss bulli
3,interview leav alon
4,son couldnt put releas alreadi bought
...,...
27475,wish could come see u denver husband lost job ...
27476,ive wonder rake client made clear net dont for...
27477,yay good enjoy break probabl need hectic weeke...
27478,worth
